## Libraries

In [1]:
import imblearn
import sklearn
import csv
print(sklearn.__version__)
print(csv.__version__)

1.2.1
1.0


In [2]:
import random

import dataset as dt
import training as tr
import preprocessing as tp
import model as m
import evaluation as eva
import utils
import config
# import nn

import pandas as pd
import numpy as np
import joblib
import datetime
import time

import json
import csv

import os
import pickle
import warnings

import sklearn
from sklearn.metrics import accuracy_score, classification_report, recall_score, precision_score#

## Settings

In [3]:
parker =  True #True #False #
# dataset names
data_index = 2
if data_index == 2 : j = 0
else: j = 1
dataset = config.datasets[data_index] #dt.Allergens
partial_key = dataset['keys'][0]
labels = dataset['labels']
feature = dataset['features'][0]
dataName = dataset['data_dir']

#LEARNING ALGORITHM
alg = config.classifier #'multinomial bayesian' #'xgboost'
#LANGUAGE MODEL
lang = config.transformers[j] #'tf-idf'
#MODEL
model_name = m.get_best_ml(data_index) #get_modelName(lang["name"], alg["name"])
print('model:', model_name)
print('---------------------------')


seed = 42

print(f'{dataName} dataset')
print('---------------------------')
print(f'Is it cleaned by Parker engine? {parker}')
print('---------------------------')
print('feature', feature)
print('---------------------------')
print('labels', labels)
print('---------------------------')

model: count-vect-xgboost
---------------------------
allergens dataset
---------------------------
Is it cleaned by Parker engine? True
---------------------------
feature ingredients
---------------------------
labels ['nuts', 'milk', 'gluten', 'soy', 'peanut', 'eggs']
---------------------------


## Evaluate ML model

#### read predicted values

In [4]:
if not parker:
    _parker = ""
    _with = "with_constraints"
else:
    _parker = "_parker"
    _with = 'with_parker'


In [5]:
configFile = f"./results/{dataset['data_dir']}/results_training_best_ml.json"

print('configFile', configFile)

f = open(configFile)

records = json.load(f)

encoder = {} 
for label in labels:
    if 'encoder' in records[_with][model_name][label]:
        encoder[label] = records[_with][model_name][label]['encoder']
    
f.close()
encoder

configFile ./results/allergens/results_training_best_ml.json


{}

#### test robustness of the trained model

#### test the trained model and save the predicted values

In [6]:
dtest = dt.read_test_csv(dataName, parker)

for a in labels:
    if a + '_gs'not in dtest.columns:
        dtest = dtest.merge(dt.read_gs_csv(dataName)[[partial_key, a ]], 
                              how='inner', on=partial_key, suffixes=('', '_gs'))

    ## load saved model
    file_model_name = f"./models/_{a}_classifier_{model_name}_{_with}.pth"
    with open(file_model_name, 'rb') as f: model = pickle.load(f)   


    # predict the values for the labels to be repaired
    enc, y_orig, y_gs = tp.encode(encoder, a, dtest)
    y_pred, outputs, dtest, accuracy = tr.clf_test(model, dtest, a, dataset, enc)

    print('a=', a, dtest[a].unique())
    # metrics
    metrics = eva.get_metrics(y_pred, y_orig.values, y_gs.values)
    print(a, 'stats: PRECISION, RECALL, F1', metrics)
    #break

print('----------------------------------------')
crs, rs, es = eva.get_all_stats(dtest, labels)
print('correct_repairs, repairs, errors', crs, rs, es)
print('precision', round(crs/rs,2))
if es !=0: 
    print('recall', round(crs/es,2))
    print('F1', 2 * round(crs/rs,2) * round(crs/es,2)/(round(crs/rs,2) + round(crs/es,2)))

relative path ./data/allergens --before delete (1635, 18)
--after delete (298, 18)
a= nuts [1 0 2]
correct repair 11 repairs 44 errors 22
nuts stats: PRECISION, RECALL, F1 (0.25, 0.5, 0.33)
a= milk [0 2 1]
correct repair 2 repairs 33 errors 6
milk stats: PRECISION, RECALL, F1 (0.06, 0.33, 0.1)
a= gluten [0 2 1]
correct repair 5 repairs 26 errors 5
gluten stats: PRECISION, RECALL, F1 (0.19, 1.0, 0.32)
a= soy [1 0 2]
correct repair 4 repairs 28 errors 7
soy stats: PRECISION, RECALL, F1 (0.14, 0.57, 0.22)
a= peanut [0 2 1]
correct repair 14 repairs 29 errors 18
peanut stats: PRECISION, RECALL, F1 (0.48, 0.78, 0.59)
a= eggs [0 2 1]
correct repair 0 repairs 12 errors 0
eggs stats: PRECISION, RECALL, F1 (0.0, 0, 0)
----------------------------------------
correct_repairs, repairs, errors 36 172 58
precision 0.21
recall 0.62
F1 0.3137349397590361


##### save last repaired datset

In [7]:
if not parker:
    a = labels[0]
    dtest[[a, a+'_gs']][dtest[a] != dtest[a+'_gs']].shape, dtest.shape
file = f"data/{dataset['data_dir']}/repaired/{dataset['data_dir']}_{model_name}_ML_repair_{_with}.csv"

print('File to be saved:', file)
#dtest.to_csv(file, quoting=csv.QUOTE_NONNUMERIC, index=False)

File to be saved: data/allergens/repaired/allergens_count-vect-xgboost_ML_repair_with_parker.csv


#### test confidence score

In [14]:
stats = {}
dtest = dt.read_test_csv(dataName, parker)
dtest1 = dtest.copy()
print(dtest1.shape)
print('+++++++++++++++++++++Start+++++++++++++++++++++++++++++')

for a in labels:
    # test repaired by parker do not have the following columns: need to fix it!!
    if a + '_gs'not in dtest1.columns:
        dtest1 = dtest1.merge(dt.read_gs_csv(dataName)[[partial_key, a ]], 
                              how='inner', on=partial_key, suffixes=('', '_gs'))
    # confidence score for each attribute
    conf_score = round(records[_with][model_name][a]['proba'],2)
    ## load saved model
    file_model_name = f"./models/_{a}_classifier_{model_name}_{_with}.pth"
    with open(file_model_name, 'rb') as f: model = pickle.load(f)   

    # get the encoder if exists and encode y_orig  y_gs
    enc = {}
    enc, y_orig, y_gs = tp.encode(encoder, a, dtest1)
    print("------ done encoding ----------")      
    
    # predict the values for the labels to be repaired
    y_pred, outputs, dtest, accuracy = tr.clf_test(model, dtest1, a, dataset, enc)
    print("------ done predicting ----------")

    if a + '_orig' not in dtest1.columns:
        dtest1 = dtest1.merge(dtest1[[partial_key, a ]], 
                              how='inner', on=partial_key, suffixes=('', '_orig')) 
        print('current columns:', dtest1.columns)

    # evaluate on ground truth
    y_repair = eva.assign_repair(outputs, y_orig.values, y_pred, conf_score)
    # stats
    correct_repair, repair, errors = eva.get_stats(y_repair, y_orig.values, y_gs.values)
    # metrics
    metrics = eva.get_metrics(y_repair, y_orig.values, y_gs.values)
    print(r'$\tau_a$', conf_score)
    print(a, 'stats: PRECISION, RECALL, F1', metrics)

    if len(encoder)>0:
        dtest1[a] = tp.decode(encoder[a], a, y_repair) #y_repair
    else: dtest1[a] = y_repair
    print("------ done replacing ----------")
    print(eva.get_stats(dtest1[a], y_orig.values, y_gs.values))

crs, rs, es = eva.get_all_stats(dtest1, labels)
print('correct_repairs, repairs, errors', crs, rs, es)
print('precision', round(crs/rs,2), 'recall', round(crs/es,2))
if es !=0 and rs!=0: 
    print('recall', round(crs/es,2))
    print('F1', 2 * round(crs/rs,2) * round(crs/es,2)/(round(crs/rs,2) + round(crs/es,2)))

relative path ./data/allergens --before delete (1635, 18)
--after delete (298, 18)
(298, 18)
+++++++++++++++++++++Start+++++++++++++++++++++++++++++
------ done encoding ----------
------ done predicting ----------
correct repair 9 repairs 19 errors 22
$\tau_a$ 0.83
nuts stats: PRECISION, RECALL, F1 (0.47, 0.41, 0.44)
------ done replacing ----------
(9, 19, 22)
------ done encoding ----------
------ done predicting ----------
correct repair 2 repairs 22 errors 6
$\tau_a$ 0.87
milk stats: PRECISION, RECALL, F1 (0.09, 0.33, 0.14)
------ done replacing ----------
(2, 22, 6)
------ done encoding ----------
------ done predicting ----------
correct repair 3 repairs 11 errors 5
$\tau_a$ 0.83
gluten stats: PRECISION, RECALL, F1 (0.27, 0.6, 0.37)
------ done replacing ----------
(3, 11, 5)
------ done encoding ----------
------ done predicting ----------
correct repair 4 repairs 17 errors 7
$\tau_a$ 0.87
soy stats: PRECISION, RECALL, F1 (0.24, 0.57, 0.34)
------ done replacing ----------
(4, 

In [9]:
file = f"./data/{dataset['data_dir']}/repaired/{dataset['data_dir']}_{model_name}_ML_repair_{_with}_threshold.csv"
print('File to be saved:', file, 'test data', dtest.shape)
file2 = f"data/{dataset['data_dir']}/{dataset['data_dir']}.csv"

dtest1.to_csv(file, quoting=csv.QUOTE_NONNUMERIC, index=False)

# ./data/allergens/repaired/allergens_count-vect-xgboost_ML_repair_with_parker_threshold.csv
file

File to be saved: ./data/allergens/repaired/allergens_count-vect-xgboost_ML_repair_with_parker_threshold.csv test data (298, 30)


'./data/allergens/repaired/allergens_count-vect-xgboost_ML_repair_with_parker_threshold.csv'

### test different thresholds

In [10]:
stats = {}
dtest = dt.read_test_csv(dataName, parker)

print('+++++++++++++++++++++Start+++++++++++++++++++++++++++++')

dtest1 = dtest.copy()
print(dtest1.shape)

for a in labels:
    # test repaired by parker do not have the following columns: need to fix it!!
    if a + '_gs'not in dtest1.columns:
        dtest1 = dtest1.merge(dt.read_gs_csv(dataName)[[partial_key, a ]], 
                              how='inner', on=partial_key, suffixes=('', '_gs'))


    avg_proba = round(records[_with][model_name][a]['proba'],2)

    thresholds = [0, avg_proba] + [th for th in np.arange(0, 1.1, 0.2)]

    ## load saved model
    file_model_name = f"./models/_{a}_classifier_{model_name}_{_with}.pth"
    with open(file_model_name, 'rb') as f: model = pickle.load(f)   

    thresholds.sort()
    print('label',a, 'avg proba', avg_proba, 'ths', thresholds)

    # get the encoder if exists and encode y_orig  y_gs
    enc = {}
    enc, y_orig, y_gs = tp.encode(encoder, a, dtest1)
    print("------ done encoding ----------")      
    
    # predict the values for the labels to be repaired
    y_pred, outputs, dtest, accuracy = tr.clf_test(model, dtest1, a, dataset, enc)
    print("------ done predicting ----------")

    if a + '_orig' not in dtest1.columns:
        dtest1 = dtest1.merge(dtest1[[partial_key, a ]], 
                              how='inner', on=partial_key, suffixes=('', '_orig')) 
        print('current columns:', dtest1.columns)
    
    repairs = []
    correct_repairs = []
    precisions = []
    recalls = []
    f1s = []

    # evaluate on ground truth
    for th in  thresholds: 
        y_repair = eva.assign_repair(outputs, y_orig.values, y_pred, th)
        # stats
        correct_repair, repair, errors = eva.get_stats(y_repair, y_orig.values, y_gs.values)
        correct_repairs.append(correct_repair)
        repairs.append(repair)

        # metrics
        metrics = eva.get_metrics(y_repair, y_orig.values, y_gs.values)
        recalls.append(round(metrics[1],2))
        precisions.append(round(metrics[0],2))
        f1s.append(round(metrics[2],2))

        print(' th', th, )
        print('stats: correct_repairs, repairs, errors', metrics)

    stats[a] = {"errors": errors, "avg_proba": avg_proba,\
                "threshold": [round(th,2) for th in thresholds], 'repairs': repairs, 'correct_repairs': correct_repairs, "precision": precisions, "recall": recalls, "F-1": f1s}
    print(f"+++++++++++++++++++++done with {a}+++++++++++++++++++++++++++++")
    print()
#         break
print('+++++++++++++++++++++more sources+++++++++++++++++++++++++++++')
print()

relative path ./data/allergens --before delete (1635, 18)
--after delete (298, 18)
+++++++++++++++++++++Start+++++++++++++++++++++++++++++
(298, 18)
label nuts avg proba 0.83 ths [0, 0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 0.83, 1.0]
------ done encoding ----------
------ done predicting ----------
correct repair 11 repairs 44 errors 22
 th 0
stats: correct_repairs, repairs, errors (0.25, 0.5, 0.33)
correct repair 11 repairs 44 errors 22
 th 0.0
stats: correct_repairs, repairs, errors (0.25, 0.5, 0.33)
correct repair 11 repairs 44 errors 22
 th 0.2
stats: correct_repairs, repairs, errors (0.25, 0.5, 0.33)
correct repair 11 repairs 44 errors 22
 th 0.4
stats: correct_repairs, repairs, errors (0.25, 0.5, 0.33)
correct repair 10 repairs 38 errors 22
 th 0.6000000000000001
stats: correct_repairs, repairs, errors (0.26, 0.45, 0.33)
correct repair 9 repairs 20 errors 22
 th 0.8
stats: correct_repairs, repairs, errors (0.45, 0.41, 0.43)
correct repair 9 repairs 19 errors 22
 th 0.83
stats: co

##### save statistics of models performences

In [11]:
statFile = f"./results/{dataset['data_dir']}/{dataset['data_dir']}_stats_{model_name}_{_with}.json"
print(statFile)
with open(statFile, "w") as outfile: 
        json.dump(stats, outfile)

./results/allergens/allergens_stats_count-vect-xgboost_with_parker.json


# End